<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* puddle_wafer(df)：

  作用：在一個DataFrame中模擬水坑（或者稱為水池）的效果。
  
  功能：根據DataFrame中的非空值的分佈，找到坑的坑底位置並將其標記為1。

  ("水坑"（Puddle）通常指的是圖像中的某個區域，該區域具有較低的亮度（或顏色值），並且周圍的區域較亮。這種區域看起來像是水坑或池塘，因為它們通常比周圍的區域更暗。)


* add_margin(df)：

  功能：對於每個column，找到第一個非空值的位置，然後在該位置的上方三行填充1。

* padding_right_margin(df, max_length)：
  
  功能：找到DataFrame中最右邊的非空column，然後擴展該column的右邊以達到指定的最大長度。

In [ ]:
## 找到坑的坑底位置並將其標記為1。##
def puddle_wafer(df):
  ground_ls = [] # 用來存儲每個column中的坑底位置（即第一個非空值的索引）
  for col in df.columns:
    for idx in df.index:
      tmp = df.at[idx, col]
      if not pd.isna(tmp): # 查tmp是否為NaN（空值）。如果不是NaN，表示找到了非空值。
        ground_ls.append(idx) # 將非空值的索引idx儲存進ground_ls
        break # 在找到每個column的第一個非空值後，中斷遍歷該column的行，以節省運算時間
      if idx == len(df): # 找到最後一個索引還是nan，則將其加入倒list
        ground_ls.append(idx)
        break
  puddle_dict = dict()
  for col_minus1 in range(1, len(ground_ls)-1): # 遍歷[第二個元素, 倒數第二個元素]
    left_min = min(ground_ls[:col_minus1]) # 找出col_minus1左側所有元素的最小值
    right_min = min(ground_ls[col_minus1+1:]) # 找出col_minus1右側所有元素的最小值
    center = ground_ls[col_minus1]
    # 檢查如果左側&右側的最小值（left/right_min）都小於當前列的索引（center），則認為這一列的左右兩側都有深色像素，表示可能存在水坑。
    if (left_min < center) & (right_min < center):
      puddle_dict[col_minus1+1] = max([left_min, right_min])
  for key in puddle_dict:
    df.at[puddle_dict[key]-1, key] = 1
  return df


## 增加邊緣 : 找出df的每一column中的 "第一個非空值", 並將其上方三行填充為1。##
def add_margin(df):
  # 創建一個空的字典margin_dict，用來儲存每個column的第一個非空值的索引。
  margin_dict = dict()
  for col in df.columns:
    for idx in df.index:
      tmp = df.at[idx, col]
      if not pd.isna(tmp): # 如果不是NaN，表示找到了非空值。
        margin_dict[col] = idx # 將非空值的索引idx存儲在margin_dict字典中，以col名作為鍵。
        break  # 在找到每個column的第一個非空值後，中斷遍歷該column的行，以節省運算時間
  '''
  在此for循環之後，margin_dict字典將包含每個column的第一個非空值的索引。
  '''

  # padding margin : 填充邊緣值
  for col in margin_dict:
    # 檢查第一個非空值的位置是否等於1。如果是1，表示需要在這個位置的"上方"填充3個1。
    if df.at[margin_dict[col], col]==1:
      df.at[margin_dict[col]-1, col]=1
      df.at[margin_dict[col]-2, col]=1
      df.at[margin_dict[col]-3, col]=1
    else:  # 如果第一個非空值的位置不是1，表示需要在這個位置的"上方"填充4個1（總共填充4行）。
      df.at[margin_dict[col]-1, col]=1
      df.at[margin_dict[col]-2, col]=1
      df.at[margin_dict[col]-3, col]=1
      df.at[margin_dict[col]-4, col]=1

  return df



## 填充其(右側)邊界以達到指定的最大長度 ##
def padding_right_margin(df, max_length):
  cols = list(df.columns)
  cols.reverse() # 將col順序倒過來
  for col in cols:
    # 檢查當前column中是否有值等於1(填充邊緣的元素)。如果是，表示找到了包含1的column (最"右邊"(外側)的非空column。)。
    if sum(df[col]==1) != 0:
      right_border_col = col # 將找到的最右邊(外側)的非空column名稱(只有"一個")，並將其存儲在right_border_col變數中。
      break
  index_ls = df[df[right_border_col]==1].index # 找到right_border_col中值為1的索引，存儲在index_ls中。
  # 計算需要添加的填充值1的"次數"。
   # (計算right_border_col中值為1的總數減去max_length的差，然後除以8並向上取整，確保不超過指定的最大長度。)
  padding_count = np.ceil((sum(df[right_border_col]==1) - max_length)/8)
  # 在特定位置填充1。
  for c in range(1, int(padding_count)+1):
    # 在距離最右邊的非空column c個位置，並且在index_ls中的相應範圍內填充。可擴展右邊邊界以達到指定的最大長度。
    df.loc[min(index_ls)+4*c : max(index_ls)-4*c, right_border_col+c] = 1
  return df



* 繪製出圖片

In [ ]:
def get_map(x, y, color, margin='circle'):
  '''
  margin = 'circle' | 'normal' | 'none'
  '''
  margin = margin.lower()
  assert margin in ['circle', 'normal', 'none'], 'margin type error'
  # padding margin --------------------------------
  # convert to matrix
  martix = np.empty([max(y)+10, max(x)+10]) # 創建一個空的矩陣matrix，維度為[max(y)+10, max(x)+10]
  matrix.fill(np.nan) #  將整個矩陣填充為 NaN 值。這表示初始時，整個圖形都是空白的。
  # set color_to_float_dict
  argmax_color_index = np.argmax(color) # 從 color 中找出最大值的"索引（位置）"
  # 這個字典將用於將顏色值映射為浮點數值。並找到顏色列表中"最大值的索引"，將其對應的浮點數值設置為0。
  color_to_float_dict = {color[argmax_color_index] : 0} #
  # 這個列表將用於將浮點數值映射回顏色。
  # 包含兩個元素。第一個元素是 0，對應到顏色列表中最大的顏色，第二個元素是 1，對應到固定的顏色 'rgb(53,160,160)'。
  float_to_color_dict = [[0, color[argmax_color_index]], [1, 'rgb(53,160,160)']]

  color_set = list(set(color)) # 創建一個包含所有不同顏色的列表color_set
  color_set.remove(color[argmax_color_index]) # 然後刪除其中 （color[argmax_color_index]）
  float_ls = list(np.linspace(0, 0.9, len(color_set)+1)) # 將set中的顏色數量均勻分佈在0到0.9之間(以便每個顏色對應到一個浮點數值)
  float_ls.remove(0) # 排除0
  # 迭代 color_set 和 float_ls 中的元素，將每種顏色映射到一個浮點數值
  for i, j in zip(color_set, float_ls):
    color_to_float_dict[i] = j
    float_to_color_dict.append([j, i])
  # "排列"。這樣可以確保顏色映射的順序是根據浮點數值的大小而定的。
  float_to_color_dict = sort(float_to_color_dict, key = lambda x: x[0]) # 按照元素的第一個值（即浮點數值）來進行排序。

  # print(float_to_color_dict)
  for i, j, c in zip(x, y, color):
    matrix[j+4, i+4] = color_to_float_dict[c]
  # convert to dataframe (並重新設置了 DataFrame 的列和行的索引，確保它們從 1 開始編號，以匹配矩陣的座標系統。)
  df = pd.DataFrame(matrix)
  df.columns = list(range(1, len(df.columns)+1))
  df.index = list(range(1, len(df)+1))
  # 1. padding upper margin
  df = add_margin(puddle_wafer(df))
  df = df.iloc[::-1].reset_index(drop = True)  #反轉 DataFrame 的row順序(上下顛倒)，然後重新設置索引(+=1)
  df.index += 1
  # 2. padding lower margin
  df = add_margin(puddle_wafer(df)).iloc[::-1].reset_index(drop = True)
  df.index += 1

  # Transpose
  df = df.T # 將 DataFrame 進行轉置，以便進行左邊緣的填充。
  # 3. padding left margin
  df = add_margin(puddle_wafer(df))
  df = df.iloc[::-1].reset_index(drop = True)
  df.index += 1
  # 4. padding right margin
  df = add_margin(puddle_wafer(df))
  df = df.iloc[::-1].reset_index(drop = True)
  df.index += 1

  df = df.T # 再次將 DataFrame 進行轉置，以恢復正確的方向。
  # finish padding
  df = df.replace({0: 'zero'}) # 將 DataFrame 中的所有零值替換為字符串 'zero'，這是用於後續圖形繪製的顏色映射的一部分。

  # ------------------------------------------------------------
  # 進行進一步的邊緣填充，以確保圖形的四個邊都具有一定的邊緣空白，使圖形看起來更加均勻
  # 右邊
  df = padding_right_margin(df, 50)
  # 左邊 : 將 DataFrame 的列進行反轉，以處理左邊的填充。這樣，原來的右邊變成了左邊，同時更新了列名的順序。
  df = df[df.columns[::-1]]
  df.columns = df.columns[::-1]
  df = padding_right_margin(df, 50)
  df = df[df.columns[::-1]] # 再次反轉列順序，恢復原始方向。
  df.columns = df.columns[::-1]
  # 下面 : 將 DataFrame 進行轉置，以便對底部進行填充。
  df = df.T
  df = padding_right_margin(df, 50)
  df = df.T # 再次反轉列順序，恢復原始方向。
  # 上面
  df = df[::-1] # 反轉 DataFrame 的row順序，以處理上邊的填充。
  df.index = range(1, len(df)+1)
  df = df.T
  df = padding_right_margin(df, 50)
  df = df.T
  df = df[::-1]
  df.index = range(1, len(df)+1)

  # to sparse matrix -------------------------
  # 將 DataFrame df 轉換為稀疏矩陣，使用 COO (Coordinate List) 格式表示。稀疏矩陣的每個元素包含三個值：行索引、列索引和數據值。
  a = sparse.coo_matrix(df)
  x = []
  y = []
  data = []
  for r, c, v in zip(a.row, a.col, a.data):
    if not pd.isna(v):
      x.append(c)
      y.append(r)
      if v == 'zero':
        v = 0
      data.append(v)

  # setting cricle -----------------------------
  # 設置用於圓形邊緣的坐標，以便在後續繪製圖形時使用。
  mid_x = (max(x)+min(x))/2 # x 軸的中心點坐標
  mid_y = abs((max(y)+min(y))/2 - max(y)) #  y 軸的中心點坐標
  r_x = (max(x)-min(x))/2 # x 半徑
  r_y = (max(y)-min(y))/2 # y 半徑
  # 計算圓的邊緣坐標 border_x 和 border_y：
  # np.arange(0, math.pi*2+0.1, 0.1) : 創建一個包含 0 到 2π 的數值範圍，間隔為 0.1。
  # np.cos 和 np.sin 函數分別計算這些角度對應的 x 和 y 方向上的坐標，並將它們乘以半徑 r_x 和 r_y。
  # 將 x 坐標的結果加上中心點坐標 mid_x，並將 y 坐標的結果加上中心點坐標 mid_y，以獲得圓的邊緣坐標。
  border_x = np.cos(np.arange(0, math.pi*2+0.1, 0.1))*r_x + mid_x
  border_y = np.sin(np.arange(0, math.pi*2+0.1, 0.1))*r_y + mid_y

  # draw figure ------------------------------------
  if margin == 'none':
    # 將 float_to_color_dict 中的最後一個元素（即最大值對應的顏色）替換為白色（'rgb(255,255,255)'）
    float_to_color_dict = float_to_color_dict[:-1] + [[1, 'rgb(255,255,255)']]
  fig = go.Figure(go.Heatmap(x=x, y=abs(np.array(y)-max(y)), z=data, colorscale=float_to_color_dict))
  if margin == 'circle':
    # 向圖形 fig 中添加一個 go.Scatter 對象，用於繪製圓形邊緣。x 和 y 是邊緣的坐標，line 參數指定線的顏色和寬度。
    fig.add_trace(go.Scatter(x=border_x, y=border_y, line=dict(color='rgb(53,160,160)', width=4)))
  # 設置 y 和 x 軸的範圍，確保圖形顯示的區域包含圓形邊緣的範圍。
  fig.update_yaxes(range=[np.min(border_y), max(border_y)])
  fig.update_xaxes(range=[np.min(border_x), max(border_x)])
  # 設置圖形的佈局和其他屬性，包括軸的顯示、背景顏色、尺寸等。
  fig.update_layout(
      xaxis = dict(showticklabels=False),
      yaxis = dict(showticklabels=False),
      coloraxis_showscale=False,
      width=250,
      height=250,
      margin = dict(autoexpand=False, l=0, b=0, r=0, t=0),
      plot_bgcolor='white',
      modebar_activecolor='rgb(152, 21, 27)'
  )

  return fig

* 保存圖片

In [ ]:
def get_map_png(x, y, color, margin='circle', size=250):
  fig = get_map(x, y, color, margin=margin)
  img = io.BytesIO(fig.to_image()) # 將圖像轉換為位元組流。 (可以添加format='png')
  img = Image.open(img).resize((size,size)) # 打開位元組流中的圖像，並將其調整大小為指定的size。
  return img

'''
import io
from PIL import Image
import plotly.graph_objs as go

def get_map_png(x, y, color, margin='circle', size=250):
    fig = get_map(x, y, color, margin=margin)
    img = fig.to_image(format="png", width=size, height=size)
    img = Image.open(io.BytesIO(img))
    return img
'''

def save_map(x, y, color, output_path, fail_path):
  assert '.png' in output_path, 'output_path_error'
  if '/' in output_path:
    folder_path = '/'.join(output_path.split('/')[:-1])
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
  img = get_map_png(x, y, color)
  img.save(output_path)
